In [1]:
import requests
import pprint 
import pandas as pd
import time 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
# import json
# import undetected_chromedriver as uc
import datetime 
import os, re
from tqdm import tqdm  # 增加进度条

In [2]:
"""
1. 请求获取酒店名和poi列表
2. 通过poi获取评分
3. 用poi关联酒店名和评分
4. 本地命令行指定端口启动浏览器，并登陆
5. 模拟点击，循环下载每个门店的评论明细（默认近一个月）
6. 处理本地评论文件，生成目标文件
"""

'\n1. 发送请求：确定url / 请求方式 / 请求头参数\n2. 获取数据\n3. 解析数据\n4. 保存数据\n5. 多页爬取\n'

In [176]:
# 获取网页所有网鱼电酒名
url = 'https://eb.meituan.com/api/v1/ebooking/comments/poiListInfo'
data = {
    '_mtsi_eb_u':'133096014'
    ,'_mtsi_eb_p':'4521183'
    ,'optimus_uuid':'4521183'
    ,'optimus_risk_level':'71'
    ,'optimus_code':'10'
}
# User-Agent 浏览器基本信息
# Referer 防盗链 告知从哪个url来
cookie = '_lxsdk_cuid=17c6efb4f3ec8-04cf88dfa26fb-b7a1438-16e360-17c6efb4f3ec8; uuid=69cc12af7cda55e1b6e7.1654154115.1.0.0; iuuid=6B9371BA2FEF3AAB8194195CA5B363C5186CB8CD61C1C253FD8D4BD0E05DD444; _lxsdk=6B9371BA2FEF3AAB8194195CA5B363C5186CB8CD61C1C253FD8D4BD0E05DD444; _hc.v=52cd0dcf-f316-55ca-e5dd-5782176ab4b8.1659681243; webp=1; __utmz=74597006.1659681549.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); mtcdn=K; __utma=74597006.966084556.1659681549.1659681549.1659926148.2; latlng=31.09843,121.316784,1659926945519; i_extend=C_b0E058951738402655627691509518768438053074_e-2557334217168929217_a网鱼电竞酒店GimthomepagesearchH__a; _ga=GA1.1.1766870804.1659938032; _gw_ab_call_5491_208=TRUE; _gw_ab_5491_208=855; WEBDFPID=v691u98v2z965v930uy9u33405x104uy81719z1v30y9795820yvzwv6-1660102329145-1660015926911UOMUMYQfd79fef3d01d5e9aadc18ccd4d0c95071427; e_u_id_3299326472=7723271a6387bf6f5069562a9e3014a9; com.sankuai.meishi.fe.kdb-bsid=GBqiaZGHSszxX2sM71k6TSyryLgl3yI7_cbeRkrBN1PO2HvGZWzP3vSGDk0na8IhKN79ykINxOlaa42oWO54NQ; _lx_utm=utm_source=Baidu&utm_medium=organic; _ga_95GX0SH5GM=GS1.1.1660019002.3.1.1660020358.0; _gw_ab_call_22454_20=TRUE; _gw_ab_22454_20=331; lt=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; u=109611883; n=mark_ppppy; token2=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; unc=mark_ppppy; IJSESSIONID=node01xem6ri5y5w83pfngpuej69pe129983641; isid=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; oops=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; mt_c_token=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; logintype=normal; cityname=上海; ci=1; rvct=1,10,20; firstTime=1660030960037; ebbsid=zEoqwRg573hTpmokx4chNaYBJUrj2_NgW3rZ56rhqt-fwg63GGMlM-jxlgaAs6tlA6UsgW3HvhAUr6JGlfsarQ; _dxuserid=zEoqwRg573hTpmokx4chNaYBJUrj2_NgW3rZ56rhqt-fwg63GGMlM-jxlgaAs6tlA6UsgW3HvhAUr6JGlfsarQ; logan_custom_report=; _yoda_verify_resp=a1F3qCTVOd49wSec+BAcFn1Ni7caK5N+qc6l5+IgQEmwztckoXHZVqG475airBHD0gpkM+zdh1t0JdkgwHi7STCM4ogKZIz06iv7GsPmjTK93e3y+D6c9OBruU1FZ/NFfIAwlQWWZF85OduLfCy+Mo1fGdn7qYE1P1In+esfh6/SVSDJVKBX3xPChBDfANsu20qUw82JRToCqvbz40aAyYAnOg4QKLY2dm7oaWZ02ryC5DODLCefaul310zrYxLoxn+CrWel1+ETgr+vu9+1jvv04oY8z0jceG2iZ94w8Agm692T5K6OtwFfrqWpwZ9T4XycFvDHcSlm0C1e1IxqfuznFqhIxjNYrsJomNbJ8/CLKwvxnKLsIWOl463ADFeG; _yoda_verify_rid=159b4d166202100a; logan_session_token=n5h6hbtw5smd4oi4qrsd; _lxsdk_s=182811db8cb-d90-724-b3c||638'
cookie_en = cookie.encode('UTF-8').decode('latin-1')
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    , 'Referer': 'https://eb.meituan.com/ebk/feedback/feedback.html'
    , 'Cookie': cookie_en

}




In [187]:
response = requests.get(url=url, params=data, headers=headers)
# pprint.pprint(response.json())
result_shopname = response.json()['data']
shopname_list = []
for shop_info in result_shopname:
    dct = {}
    dct['shopname'] = shop_info['poiName']
    dct['poiIdStr'] = shop_info['poiIdStr']
    shopname_list.append(dct)
df_shopname = pd.DataFrame(shopname_list)
# print(response.json())

In [179]:
def get_mt_score(poiIdStr: str, platform: str):
    """根据传入的poi和平台标志获取对应门店评分 platform: 1美团 0点评"""
    url_com = 'https://eb.meituan.com/api/v1/ebooking/common/poi/%s/contrast'%poiIdStr
    # todo 这里从浏览器复制出来后可以用正则替换成字典格式
    data_com = {
        'platform': platform,
        '_mtsi_eb_u': '133096014',
        '_mtsi_eb_p': '4521183',
        'optimus_uuid': '4521183',
        'optimus_risk_level': '71',
        'optimus_code': '10'
    }
    
    # User-Agent 浏览器基本信息
    # Referer 防盗链 告知从哪个url来
    cookie_com = '_lxsdk_cuid=17c6efb4f3ec8-04cf88dfa26fb-b7a1438-16e360-17c6efb4f3ec8; uuid=69cc12af7cda55e1b6e7.1654154115.1.0.0; iuuid=6B9371BA2FEF3AAB8194195CA5B363C5186CB8CD61C1C253FD8D4BD0E05DD444; _lxsdk=6B9371BA2FEF3AAB8194195CA5B363C5186CB8CD61C1C253FD8D4BD0E05DD444; _hc.v=52cd0dcf-f316-55ca-e5dd-5782176ab4b8.1659681243; webp=1; __utmz=74597006.1659681549.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); mtcdn=K; __utma=74597006.966084556.1659681549.1659681549.1659926148.2; latlng=31.09843,121.316784,1659926945519; i_extend=C_b0E058951738402655627691509518768438053074_e-2557334217168929217_a网鱼电竞酒店GimthomepagesearchH__a; _ga=GA1.1.1766870804.1659938032; _gw_ab_call_5491_208=TRUE; _gw_ab_5491_208=855; WEBDFPID=v691u98v2z965v930uy9u33405x104uy81719z1v30y9795820yvzwv6-1660102329145-1660015926911UOMUMYQfd79fef3d01d5e9aadc18ccd4d0c95071427; e_u_id_3299326472=7723271a6387bf6f5069562a9e3014a9; com.sankuai.meishi.fe.kdb-bsid=GBqiaZGHSszxX2sM71k6TSyryLgl3yI7_cbeRkrBN1PO2HvGZWzP3vSGDk0na8IhKN79ykINxOlaa42oWO54NQ; _lx_utm=utm_source=Baidu&utm_medium=organic; _ga_95GX0SH5GM=GS1.1.1660019002.3.1.1660020358.0; _gw_ab_call_22454_20=TRUE; _gw_ab_22454_20=331; lt=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; u=109611883; n=mark_ppppy; token2=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; unc=mark_ppppy; IJSESSIONID=node01xem6ri5y5w83pfngpuej69pe129983641; isid=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; oops=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; mt_c_token=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; logintype=normal; cityname=上海; ci=1; rvct=1,10,20; firstTime=1660030960037; ebbsid=zEoqwRg573hTpmokx4chNaYBJUrj2_NgW3rZ56rhqt-fwg63GGMlM-jxlgaAs6tlA6UsgW3HvhAUr6JGlfsarQ; _dxuserid=zEoqwRg573hTpmokx4chNaYBJUrj2_NgW3rZ56rhqt-fwg63GGMlM-jxlgaAs6tlA6UsgW3HvhAUr6JGlfsarQ; logan_custom_report=; logan_session_token=9qdjvg9f4rhgfmvq48j0; _lxsdk_s=182811db8cb-d90-724-b3c||599'
    cookie_com_en = cookie_com.encode('UTF-8').decode('latin-1')
    headers_com = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
        , 'Referer': 'https://eb.meituan.com/ebk/feedback/feedback.html'
        , 'Cookie': cookie_com_en
        , 'Host': 'eb.meituan.com'
        , 'Accept':'application/json, text/plain, */*'
        , 'Accept-Encoding': 'gzip, deflate, br'
        , 'Accept-Language': 'zh-CN,zh;q=0.9'
        , 'Connection': 'keep-alive'
        , 'mtgsig': '{"a1":"1.0","a2":1660032455481,"a3":"v691u98v2z965v930uy9u33405x104uy81719z1v30y9795820yvzwv6","a4":"b8c42682e43d35438226c4b843353de4bb356139f8005cdd","a5":"nPoyiBTRD+xO69/F5gpbhdWjmYnu0HvDnoPuOiK+zGuYHWsMRh8HMHyxMIOqGT6DGNPy3PJrg1tUO/LEnKombhpMSqth","a6":"h1.1fRNzeRP4ouP95ylmc6N6uxwxReMyC1CLo9Iw0Yy4QtGQQ6H5ArCTxnvBNDyS+vhO/jolCWraeiVOu+jcHudahlrRVC6I0ew6OM8Ao0GK7E7ohiyT2Me8WH4rQd35ChbaXa1QGdlAQkhGkOKCgrNpvXlA+VdV9QahaIYe29psD5nXoKshWN3W3CGYyYVxqBeJEMFG2sOonoc5Vnr4Vfu8ibW2PnHLZ17uW5fkJxLtbmvFEMaMesWwxThZ92smTrymQblmhJwUSb/i5MuOxqFddZd4JyEsh+0Zvd5DdpbTz8M0vIhPVUIIEioVBKMU0vxMdnMx8Ycm+ihlTPWje4wiwICEFZZ4zSNUjse4pybpeL/7QweNb5eMmQU/uhc7kBjesbwJmxkw67H3ppETE/7OMfTd8bUJRD+rWG8Bm03E2YQb/DTberx2skYo0BmLk69/0Xrga2q4M/wAwWauLRdo6Q==","a7":"","x0":4,"d1":"5e432b79a56d45c9cc698e152206d3c8"}'
        , 'sec-ch-ua': '"Chromium";v="104", " Not A;Brand";v="99", "Google Chrome";v="104"'
        , 'sec-ch-ua-mobile': '?0' 
        , 'sec-ch-ua-platform': "Windows"
        , 'Sec-Fetch-Dest': 'empty'
        , 'Sec-Fetch-Mode': 'cors'
        , 'Sec-Fetch-Site': 'same-origin'
    }
    response_com = requests.get(url=url_com, params=data_com, headers=headers_com)
    try: 
        score = response_com.json()['data']['score'] / 10
    except:
        score = 0
        print(response_com.json())  # 有时候需要手动验证

    time.sleep(1)

    return score

In [216]:
# 获取报错时，更换门店POI，单独执行这段代码，获取验证链接，然后执行下面的代码就不会错了
# """根据传入的poi和平台标志获取对应门店评分 platform: 1美团 0点评"""
# url_com = 'https://eb.meituan.com/api/v1/ebooking/common/poi/%s/contrast'%'922833861'
# # todo 这里从浏览器复制出来后可以用正则替换成字典格式
# data_com = {
#     'platform': '1',
#     '_mtsi_eb_u': '133096014',
#     '_mtsi_eb_p': '4521183',
#     'optimus_uuid': '4521183',
#     'optimus_risk_level': '71',
#     'optimus_code': '10'
# }

# # User-Agent 浏览器基本信息
# # Referer 防盗链 告知从哪个url来
# cookie_com = '_lxsdk_cuid=17c6efb4f3ec8-04cf88dfa26fb-b7a1438-16e360-17c6efb4f3ec8; uuid=69cc12af7cda55e1b6e7.1654154115.1.0.0; iuuid=6B9371BA2FEF3AAB8194195CA5B363C5186CB8CD61C1C253FD8D4BD0E05DD444; _lxsdk=6B9371BA2FEF3AAB8194195CA5B363C5186CB8CD61C1C253FD8D4BD0E05DD444; _hc.v=52cd0dcf-f316-55ca-e5dd-5782176ab4b8.1659681243; webp=1; __utmz=74597006.1659681549.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); mtcdn=K; __utma=74597006.966084556.1659681549.1659681549.1659926148.2; latlng=31.09843,121.316784,1659926945519; i_extend=C_b0E058951738402655627691509518768438053074_e-2557334217168929217_a网鱼电竞酒店GimthomepagesearchH__a; _ga=GA1.1.1766870804.1659938032; _gw_ab_call_5491_208=TRUE; _gw_ab_5491_208=855; WEBDFPID=v691u98v2z965v930uy9u33405x104uy81719z1v30y9795820yvzwv6-1660102329145-1660015926911UOMUMYQfd79fef3d01d5e9aadc18ccd4d0c95071427; e_u_id_3299326472=7723271a6387bf6f5069562a9e3014a9; com.sankuai.meishi.fe.kdb-bsid=GBqiaZGHSszxX2sM71k6TSyryLgl3yI7_cbeRkrBN1PO2HvGZWzP3vSGDk0na8IhKN79ykINxOlaa42oWO54NQ; _lx_utm=utm_source=Baidu&utm_medium=organic; _ga_95GX0SH5GM=GS1.1.1660019002.3.1.1660020358.0; _gw_ab_call_22454_20=TRUE; _gw_ab_22454_20=331; lt=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; u=109611883; n=mark_ppppy; token2=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; unc=mark_ppppy; IJSESSIONID=node01xem6ri5y5w83pfngpuej69pe129983641; isid=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; oops=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; mt_c_token=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; logintype=normal; cityname=上海; ci=1; rvct=1,10,20; firstTime=1660030960037; ebbsid=zEoqwRg573hTpmokx4chNaYBJUrj2_NgW3rZ56rhqt-fwg63GGMlM-jxlgaAs6tlA6UsgW3HvhAUr6JGlfsarQ; _dxuserid=zEoqwRg573hTpmokx4chNaYBJUrj2_NgW3rZ56rhqt-fwg63GGMlM-jxlgaAs6tlA6UsgW3HvhAUr6JGlfsarQ; logan_custom_report=; logan_session_token=9qdjvg9f4rhgfmvq48j0; _lxsdk_s=182811db8cb-d90-724-b3c||599'
# cookie_com_en = cookie_com.encode('UTF-8').decode('latin-1')
# headers_com = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
#     , 'Referer': 'https://eb.meituan.com/ebk/feedback/feedback.html'
#     , 'Cookie': cookie_com_en
#     , 'Host': 'eb.meituan.com'
#     , 'Accept':'application/json, text/plain, */*'
#     , 'Accept-Encoding': 'gzip, deflate, br'
#     , 'Accept-Language': 'zh-CN,zh;q=0.9'
#     , 'Connection': 'keep-alive'
#     , 'mtgsig': '{"a1":"1.0","a2":1660032455481,"a3":"v691u98v2z965v930uy9u33405x104uy81719z1v30y9795820yvzwv6","a4":"b8c42682e43d35438226c4b843353de4bb356139f8005cdd","a5":"nPoyiBTRD+xO69/F5gpbhdWjmYnu0HvDnoPuOiK+zGuYHWsMRh8HMHyxMIOqGT6DGNPy3PJrg1tUO/LEnKombhpMSqth","a6":"h1.1fRNzeRP4ouP95ylmc6N6uxwxReMyC1CLo9Iw0Yy4QtGQQ6H5ArCTxnvBNDyS+vhO/jolCWraeiVOu+jcHudahlrRVC6I0ew6OM8Ao0GK7E7ohiyT2Me8WH4rQd35ChbaXa1QGdlAQkhGkOKCgrNpvXlA+VdV9QahaIYe29psD5nXoKshWN3W3CGYyYVxqBeJEMFG2sOonoc5Vnr4Vfu8ibW2PnHLZ17uW5fkJxLtbmvFEMaMesWwxThZ92smTrymQblmhJwUSb/i5MuOxqFddZd4JyEsh+0Zvd5DdpbTz8M0vIhPVUIIEioVBKMU0vxMdnMx8Ycm+ihlTPWje4wiwICEFZZ4zSNUjse4pybpeL/7QweNb5eMmQU/uhc7kBjesbwJmxkw67H3ppETE/7OMfTd8bUJRD+rWG8Bm03E2YQb/DTberx2skYo0BmLk69/0Xrga2q4M/wAwWauLRdo6Q==","a7":"","x0":4,"d1":"5e432b79a56d45c9cc698e152206d3c8"}'
#     , 'sec-ch-ua': '"Chromium";v="104", " Not A;Brand";v="99", "Google Chrome";v="104"'
#     , 'sec-ch-ua-mobile': '?0' 
#     , 'sec-ch-ua-platform': "Windows"
#     , 'Sec-Fetch-Dest': 'empty'
#     , 'Sec-Fetch-Mode': 'cors'
#     , 'Sec-Fetch-Site': 'same-origin'
# }
# response_com = requests.get(url=url_com, params=data_com, headers=headers_com)
# response_com.json()  

In [217]:
# response_com.json()
# df_shopname[:2]

{'msg': '您的网络好像不太给力，请稍后再试',
 'code': 406,
 'customData': {'verifyPageUrl': 'https://verify.meituan.com/v2/web/general_page?action=spiderindefence&requestCode=605b8793488e4119b31a6b8546686321&platform=1&adaptor=auto&succCallbackUrl=https://optimus-mtsi.meituan.com/optimus/verifyResult',
  'riskLevel': '103',
  'imageUrl': 'https://verify.meituan.com/v2/captcha?action=spiderindefence&request_code=605b8793488e4119b31a6b8546686321',
  'requestCode': '605b8793488e4119b31a6b8546686321',
  'verifyUrl': 'https://optimus-mtsi.meituan.com/optimus/verify?request_code=605b8793488e4119b31a6b8546686321'}}

In [218]:
shop_score = []
# df_shopname = df_shopname[:1]
for shopinfo in df_shopname.to_dict('records'):
    # 获取门店评分
    mt_score = get_mt_score(shopinfo['poiIdStr'], '1')
    dp_score = get_mt_score(shopinfo['poiIdStr'], '0')
    dct = {}
    dct['poiIdStr'] = shopinfo['poiIdStr']
    dct['mt_score'] = mt_score
    dct['dp_score'] = dp_score
    shop_score.append(dct)
df_shop_score = pd.DataFrame(shop_score)
df_shop_score = pd.merge(df_shopname, df_shop_score, how='left', on='poiIdStr')

In [219]:
df_shop_score

,shopname,poiIdStr,mt_score,dp_score
0,网鱼电竞酒店（上海光大国际会展中心店）,1919978105,4.8,4.0
1,网鱼电竞酒店（普陀长寿路店）,922833861,5.0,4.0
2,网鱼电竞酒店（广州赤岗地铁站店）,1590414218,4.9,4.2
3,网鱼电竞酒店（深圳布吉东站店）,1781755197,4.9,4.4
4,网鱼电竞酒店（上海松江万达店）,192763065,5.0,3.9
5,网鱼电竞酒店（上海嘉定南翔店）,1866796500,5.0,3.7
6,网鱼电竞酒店（武汉汉口火车站店）,1182484990,5.0,4.3
7,网鱼电竞酒店（广州岗贝路店）,578091378,5.0,4.7
8,网鱼电竞酒店（上海奉贤南桥店）,1086430732,5.0,4.5
9,网鱼电竞酒店（广州东圃客运站店）,965958593,5.0,3.9


In [15]:
# 获取美团近7天评论数量 / 差评数量

# def get_driver():
#     s = Service(r"C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")
#     options = webdriver.ChromeOptions()
#     options.add_argument('lang=zh_CN.UTF-8')
#     # options.add_argument('--disable-javascript') # 禁用javascript
#     # options.add_argument('blink-settings=imagesEnabled=false')
#     options.add_argument('--start-maximized') # 最大化运行（全屏窗口）,不设置，取元素会报错
#     # options.add_argument('--disable-infobars') # 禁止策略化
#     # options.add_argument('--incognito') # 隐身模式（无痕模式）
#     # options.add_argument('--headless') # 浏览器不提供可视化页面
#     options.add_experimental_option('excludeSwitches', ['enable-automation']) 
#     driver = webdriver.Chrome(service=s,options=options)
#     return driver

# def cookie_to_dic(cookie):
#      return {item.split('=')[0]: item.split('=')[1] for item in cookie.split('; ')}

# driver = get_driver()
# url = 'https://eb.meituan.com/ebk/feedback/feedback.html#/mt'
# 未携带cookies打开网页
# driver.get(url) 

# 180s内输入账号密码，并登录
# time.sleep(10)
# driver.find_element(By.CLASS_NAME, "feedback-list__other-btns ng-scope").click()
# with open('mt_cookies.txt','w') as f:
#     # 将cookies保存为json格式
#     f.write(json.dumps(driver.get_cookies()))
# driver.close()

# cookie_com = '_lxsdk_cuid=17c6efb4f3ec8-04cf88dfa26fb-b7a1438-16e360-17c6efb4f3ec8; uuid=69cc12af7cda55e1b6e7.1654154115.1.0.0; iuuid=6B9371BA2FEF3AAB8194195CA5B363C5186CB8CD61C1C253FD8D4BD0E05DD444; _lxsdk=6B9371BA2FEF3AAB8194195CA5B363C5186CB8CD61C1C253FD8D4BD0E05DD444; _hc.v=52cd0dcf-f316-55ca-e5dd-5782176ab4b8.1659681243; webp=1; __utmz=74597006.1659681549.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); mtcdn=K; __utma=74597006.966084556.1659681549.1659681549.1659926148.2; latlng=31.09843,121.316784,1659926945519; i_extend=C_b0E058951738402655627691509518768438053074_e-2557334217168929217_a网鱼电竞酒店GimthomepagesearchH__a; _ga=GA1.1.1766870804.1659938032; _gw_ab_call_5491_208=TRUE; _gw_ab_5491_208=855; WEBDFPID=v691u98v2z965v930uy9u33405x104uy81719z1v30y9795820yvzwv6-1660102329145-1660015926911UOMUMYQfd79fef3d01d5e9aadc18ccd4d0c95071427; e_u_id_3299326472=7723271a6387bf6f5069562a9e3014a9; com.sankuai.meishi.fe.kdb-bsid=GBqiaZGHSszxX2sM71k6TSyryLgl3yI7_cbeRkrBN1PO2HvGZWzP3vSGDk0na8IhKN79ykINxOlaa42oWO54NQ; _lx_utm=utm_source=Baidu&utm_medium=organic; _ga_95GX0SH5GM=GS1.1.1660019002.3.1.1660020358.0; _gw_ab_call_22454_20=TRUE; _gw_ab_22454_20=331; lt=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; u=109611883; n=mark_ppppy; token2=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; unc=mark_ppppy; IJSESSIONID=node01xem6ri5y5w83pfngpuej69pe129983641; isid=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; oops=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; mt_c_token=13ZdDx70gtNCCIUc9Vgea6ruMSwAAAAAQRMAAGAqch7KPMQHkinsHxR6ZbTXn_HBQQy2bm4QdSEMTBb3Rw6YCE_EEBIXKiDC--yzVw; logintype=normal; cityname=上海; ci=1; rvct=1,10,20; firstTime=1660030960037; ebbsid=zEoqwRg573hTpmokx4chNaYBJUrj2_NgW3rZ56rhqt-fwg63GGMlM-jxlgaAs6tlA6UsgW3HvhAUr6JGlfsarQ; _dxuserid=zEoqwRg573hTpmokx4chNaYBJUrj2_NgW3rZ56rhqt-fwg63GGMlM-jxlgaAs6tlA6UsgW3HvhAUr6JGlfsarQ; logan_custom_report=; logan_session_token=9qdjvg9f4rhgfmvq48j0; _lxsdk_s=182811db8cb-d90-724-b3c||599'
# cookie_com_en = cookie_com.encode('UTF-8').decode('latin-1')
# cookie_com_en_dct = cookie_to_dic(cookie_com_en)
# cookie_com_en_dct['name'] = 'mt'
# cookie_com_en_dct['value'] = 'mt'
# cookie_com_en_dct['domain'] = 'eb.meituan.com'
# driver.add_cookie(cookie_com_en_dct)

# driver.refresh()


In [32]:
# 本机命令行指定端口运行浏览器
# cd C:\Program Files (x86)\Google\Chrome\Application
# chrome.exe -remote-debugging-port=9222

# 调用已经打开的浏览器  执行过程中不要操作电脑，会影响页面加载，导致运行失败
s = Service(r"C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(service=s,options=options)
print(driver.title)

# 所有门店名 
# html = driver.page_source  # 获取网页的html数据
# soup = BeautifulSoup(html, 'lxml')
# shop_html_lst = soup.find_all('li', class_='el-select-dropdown__item')
# shop_name = []
# for li in shop_html_lst:
#     shop_name.append(li.getText())

lst_shop_score = []
for i in tqdm(range(0, 48)):  # 48  这个数字是门店个数，如有新增门店，同步这个数字

    driver.find_element(By.XPATH, '//*[@id="ng-main"]/div/div/div[1]/div/div/div/a/span[3]/b').click()  # 点击下拉框箭头
    driver.find_element(By.ID, 'ui-select-choices-row-0-%d'%i).click()  # 门店列表中的门店名
    time.sleep(2)  # 等页面加载
    # 点击 美团评价 
    driver.find_element(By.LINK_TEXT, '美团评价').click()
    time.sleep(2)  # 等页面加载
    
    # 获取当前页面的门店名和美团评分 
    html_mt = driver.page_source  # 获取网页的html数据
    soup_mt = BeautifulSoup(html_mt, 'lxml')
    dct_shop_score = {}
    cur_shop_name = soup_mt.find('span', class_='ng-binding ng-scope').get_text()  # 当前门店名
    cur_shop_name = cur_shop_name.replace('\n', '').replace(' ', '')
    dct_shop_score['shopname'] = cur_shop_name
    score_mt = soup_mt.find('em', class_='hotel-summary__num ng-binding').get_text()  # 当前门店评分
    dct_shop_score['mt_score'] = score_mt
    # 下载当前门店近一个月评价
    driver.find_element(By.PARTIAL_LINK_TEXT, '全部评价').click()  # 点击 全部评价 
    time.sleep(2)  # 等页面加载
    driver.find_element(By.LINK_TEXT, '下载评价列表').click()  # 点击 下载评价列表 
    # TODO 在时间范围下拉框中键入日期
    driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-primary').click()  # 点击下载 默认下载最近一个月的评论 
    time.sleep(2)  # 等页面恢复

    # 点击 点评评价 并获取点评评分
    driver.find_element(By.LINK_TEXT, '点评评价').click() 
    time.sleep(2)  # 等页面恢复
    html_dp = driver.page_source  # 获取网页的html数据
    soup_dp = BeautifulSoup(html_dp, 'lxml')
    score_dp = soup_dp.find('em', class_='hotel-summary__num ng-binding').get_text()
    dct_shop_score['dp_score'] = score_dp

    lst_shop_score.append(dct_shop_score)


df_shop_score = pd.DataFrame(lst_shop_score)

# driver.find_element(By.XPATH, '//*[@id="ng-main"]/div/div/div[1]/div/div/div/a/span[3]/b').click()  # 点击下拉框箭头
# driver.find_element(By.ID, 'ui-select-choices-row-0-1').click()  # 门店列表中的门店名
# time.sleep(1)  # 等页面加载
# driver.find_element(By.PARTIAL_LINK_TEXT, '全部评价').click()  # 点击 全部评价 
# time.sleep(1)  # 等页面加载
# driver.find_element(By.LINK_TEXT, '下载评价列表').click()  # 点击 下载评价列表
# driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-primary').click()

美团酒店EBooking


In [33]:
# python读取下载的文件，统计每家门店的 (时间段内)评论数量、差评数量、差评内容、差评分数、商家是否回复（取不到）、差评时间

def get_range_date():
    today = datetime.date.today()  # 获取当前日期  datetime.date(2022, 8, 12)
    oneday = datetime.timedelta(days=1)  
    eightday = datetime.timedelta(days=7)
    end_date = today - oneday  
    start_date = today - eightday
    return start_date, end_date

# 批量读取路径下固定格式文件名的文件


In [34]:
file_path = 'C:/Users/118252/Downloads/'  # 反斜杠要改为正斜杠，避免被判断为转义符 
file_list = os.listdir(file_path)  # 获取目标路径下的所有文件名
file_prefix = '美团评价-'+str(datetime.date.today())+'-.*'  # 匹配的正则格式
re_pattern = re.compile(pattern=file_prefix) 

df_all = pd.DataFrame([])
for file_name in file_list:
    if re_pattern.search(file_name):
        df_sub = pd.read_excel(file_path + file_name
                               , parse_dates=['评价时间']
                               , date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d:%H:%M:%S')) 
        df_all = pd.concat([df_all, df_sub])



In [35]:
s_date, e_date = get_range_date()
df_all_screen = df_all[(df_all['评价时间'].dt.date>=s_date)&(df_all['评价时间'].dt.date<=e_date)]
# 统计每家店的店名、评论数量、差评数量
df_statistics = pd.DataFrame([])
df_statistics['评价数量'] = df_all_screen.groupby(['分店名'])['评分'].count()
df_statistics['差评数量'] = df_all_screen[df_all_screen['评分']<=3].groupby(['分店名'])['评分'].count()
df_statistics.reset_index(inplace=True)
# 取出每家店的店名、差评时间、差评内容、差评分数
df_negative = df_all_screen[df_all_screen['评分']<=3][['分店名', '评价内容', '评分', '评价时间']]
df_negative.columns = ['分店名', '差评内容', '差评星级', '差评时间']
# 将上面的两部分数据，和门店评分关联，生成excel
df_shop = pd.merge(df_shop_score, df_statistics, how='left', right_on='分店名', left_on='shopname')
df_shop = df_shop[['shopname', 'mt_score', 'dp_score', '评价数量', '差评数量']]
df_result = pd.merge(df_shop, df_negative, how='left', left_on='shopname', right_on='分店名')
df_result = df_result[['shopname', 'mt_score', 'dp_score', '评价数量', '差评数量', '差评内容', '差评星级', '差评时间']]
df_result.columns = ['门店名', '美团评分', '点评评分', '评价数量', '差评数量', '差评内容', '差评星级', '差评时间']
df_result['评价数量'].fillna(0, inplace=True)
df_result['差评数量'].fillna(0, inplace=True)
df_result.to_excel('../../data/美团点评评分和近7天评论.xlsx')